In [4]:
import matplotlib.pyplot as plt

import numpy as np
import pandas as pd

import os, sys, datetime, pickle
import cv2, csv
import math
import itertools


from glob import glob
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix,f1_score


from tensorflow.keras.utils import to_categorical
from keras.preprocessing.image import ImageDataGenerator
from keras.applications.inception_v3 import InceptionV3
from keras.applications.resnet50 import ResNet50
from keras.applications import VGG19

from keras.preprocessing import image
from keras.models import Model
from keras.layers import Dense, GlobalAveragePooling2D, Flatten, Dropout
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint

from tensorflow.keras.regularizers import l2
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import LearningRateScheduler, EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.optimizers.schedules import ExponentialDecay



Using TensorFlow backend.


In [78]:
RESULTS_Y_DIR = 'E:/DeepLearning/test5/save_y_pred'
RESULTS_HIST_DIR = 'E:/DeepLearning/test5/save_hist'
RESULTS_HIST_DIR = 'E:/DeepLearning/test5/save_list'
PREFIX_TEST = ['5_1','5_2','5_3','6_1','6_2','6_3','7_1','7_1_1','7_1_2','7_2','7_2_1','7_2_2','7_3','7_3_1','8_1_1', '9_1_1']
#PREFIX_TEST = ['8_1_1']
SIZE = [75, 128, 256, 320, 384]

TEST_FILE = 'E:/DeepLearning/test_file_list.csv'
#CLASS_NAME = ['Sugar beet',  'Maize',    'Charlock', 'FatHen',           'Cleavers',     'Loose Silky-bent',
#              'Commonwheat',     'Black-grass', 'Small-flowered Cranesbill', 'CommonChickweed', 'Scentless Mayweed', 'Shepherds Purse']
CLASS_NAME = ['Black-grass', 'Charlock', 'Cleavers', 'Common Chickweed', 'Common wheat', 'Fat Hen',
              'Loose Silky-bent', 'Maize', 'Scentless Mayweed','Shepherds Purse', 'Small-flowered Cranesbill', 'Sugar beet']


In [79]:
test_file = []
with open(TEST_FILE, 'r') as f:
    test_file = f.read().split('\n')
while '' in test_file:
    test_file.remove('')
print(len(test_file))


794


In [82]:
for pref_test in PREFIX_TEST:
    #print('TEST ' + pref_test)
    for size in SIZE:
        #print(size)
        #print(pref_test + ' ' + str(size))
        if '7_1_1' not in pref_test and '7_1_2' not in pref_test and '7_2_1' not in pref_test and '7_2' not in pref_test\
                    and '7_1' not in pref_test and '7_3' not in pref_test \
                    and '7_2_2' not in pref_test and '7_3_1' not in pref_test and '8_1_1' not in pref_test:
            print('>>>>' + pref_test)
            list_train_results = glob(RESULTS_Y_DIR + '/' + pref_test + '*' + str(size) + '*train*.csv')
            list_val_results = glob(RESULTS_Y_DIR + '/' + pref_test + '*' + str(size) + '*val*.csv')
            list_test_results = glob(RESULTS_Y_DIR + '/' + pref_test + '*' + str(size) + '*test*.csv')
            if len(list_train_results) == 1:
                print('TEST ' + pref_test)
                with open(list_train_results[0], 'r') as f:
                    res_y_train = f.read().split('\n')
                    #print(res_y_train)

                    y_true = res_y_train[0].split(',')
                    y_pred = res_y_train[1].split(',')
                    
                    f1_train = np.around(f1_score(y_true, y_pred, average='micro'),4)
                    #print('Train F1 ,'   + str(f1))                    

                with open(list_val_results[0], 'r') as f:
                    res_y_train = f.read().split('\n')
                    #print(res_y_train)

                    y_true = res_y_train[0].split(',')
                    y_pred = res_y_train[1].split(',')
                    
                    f1_valid = np.around(f1_score(y_true, y_pred, average='micro'),4)
                print(str(size) + ',' +  str(f1_train) + ',' +  str(f1_valid))                      
                    
                    
                test_file_order = glob(RESULTS_HIST_DIR + '/' + pref_test + '*' + str(size) + '*_test_file_list.csv')   
                if (len(test_file_order)):
                    with open(test_file_order[0], 'r') as f:
                        test_file = f.read().split('\n')
                    while '' in test_file:
                        test_file.remove('')
                    print(len(test_file))                
                
                    with open(list_test_results[0], 'r') as f:
                        res_y_test = f.read()
                        #print(res_y_train)
                        y_pred_test = res_y_test.split(',')
                        #print(len(y_pred_test))
                        
                        output_file = 'kaggle_' + list_test_results[0].split('\\')[-1]
                        with open(output_file, 'w') as f1:
                            f1.write('file,species\n')
                            for i in range(len(y_pred_test)):
                                f1.write(test_file[i] + ',' + CLASS_NAME[int(y_pred_test[i])] + '\n')
                    
                    
           
                hist_val_acc_files = glob(RESULTS_HIST_DIR + '/' + pref_test + '*' + str(size) + '*hist_val_acc')
                hist_accuracy_files =  glob(RESULTS_HIST_DIR + '/' + pref_test + '*' + str(size) +'*hist_accuracy')
                if len(hist_accuracy_files) != 0:
                    with open(hist_accuracy_files[0], "rb") as f:
                        hist_acc = pickle.load(f)
                        #print(hist_acc)
                if len(hist_val_acc_files) != 0:
                    with open(hist_val_acc_files[0], "rb") as f:
                        hist_val_acc = pickle.load(f)
                        #print(hist_val_acc)                            

                
        else :
            
            trial = [1,2]
            for t in trial:
                
                list_train_results = glob(RESULTS_Y_DIR + '/' + pref_test + '*unfrozen' + str(t) + '*' + str(size) + '*train*.csv')
                list_val_results = glob(RESULTS_Y_DIR + '/' + pref_test + '*unfrozen' + str(t) + '*' + str(size) + '*val*.csv')
                list_test_results = glob(RESULTS_Y_DIR + '/' + pref_test + '*unfrozen' + str(t) + '*' + str(size) + '*test*.csv') 
                if len(list_train_results):
                    print('TEST ' + pref_test + ' ' + str(t))
                    with open(list_train_results[0], 'r') as f:
                        res_y_train = f.read().split('\n')
                        #print(res_y_train)

                        y_true = res_y_train[0].split(',')
                        y_pred = res_y_train[1].split(',')

                        f1_train = np.around(f1_score(y_true, y_pred, average='micro'),4)
                        #print('Train F1 ,'   + str(f1))                    

                    with open(list_val_results[0], 'r') as f:
                        res_y_train = f.read().split('\n')
                        #print(res_y_train)

                        y_true = res_y_train[0].split(',')
                        y_pred = res_y_train[1].split(',')

                        f1_valid = np.around(f1_score(y_true, y_pred, average='micro'),4)
                    print(str(size) + ',' +  str(f1_train) + ',' +  str(f1_valid))            
                    
                    
                    
                    test_file_order = glob(RESULTS_HIST_DIR + '/' + pref_test + '*' + str(size) + '*_test_file_list.csv')   
                    if (len(test_file_order)):
                        with open(test_file_order[0], 'r') as f:
                            test_file = f.read().split('\n')
                        while '' in test_file:
                            test_file.remove('')
                        print(len(test_file))                

                        with open(list_test_results[0], 'r') as f:
                            res_y_test = f.read()
                            #print(res_y_train)
                            y_pred_test = res_y_test.split(',')
                            #print(len(y_pred_test))

                            output_file = 'kaggle_' + list_test_results[0].split('\\')[-1]
                            with open(output_file, 'w') as f1:
                                f1.write('file,species\n')
                                for i in range(len(y_pred_test)):
                                    f1.write(test_file[i] + ',' + CLASS_NAME[int(y_pred_test[i])] + '\n')
                                    
                                    
        list_train_results = glob(RESULTS_Y_DIR + '/' + '9_1_1_VGG19-trainnet-mask-flavia-imgsize320_train_ypred.csv')
        list_val_results = glob(RESULTS_Y_DIR + '/' + '9_1_1_VGG19-trainnet-mask-flavia-imgsize320_val_ypred.csv')
        if len(list_train_results):
            print('TEST ' + '9 net' + ' ' )
            with open(list_train_results[0], 'r') as f:
                res_y_train = f.read().split('\n')
                #print(res_y_train)

                y_true = res_y_train[0].split(',')
                y_pred = res_y_train[1].split(',')

                f1_train = np.around(f1_score(y_true, y_pred, average='micro'),4)
                #print('Train F1 ,'   + str(f1))                    

            with open(list_val_results[0], 'r') as f:
                res_y_train = f.read().split('\n')
                #print(res_y_train)

                y_true = res_y_train[0].split(',')
                y_pred = res_y_train[1].split(',')

                f1_valid = np.around(f1_score(y_true, y_pred, average='micro'),4)
            print(str(size) + ',' +  str(f1_train) + ',' +  str(f1_valid))    
        

>>>>5_1
TEST 5_1
75,0.9992,0.6695
TEST 9 net 
75,0.7049,0.6885
>>>>5_1
TEST 5_1
128,0.9921,0.7389
TEST 9 net 
128,0.7049,0.6885
>>>>5_1
TEST 5_1
256,0.9971,0.7463
TEST 9 net 
256,0.7049,0.6885
>>>>5_1
TEST 9 net 
320,0.7049,0.6885
>>>>5_1
TEST 5_1
384,0.975,0.6663
TEST 9 net 
384,0.7049,0.6885
>>>>5_2
TEST 5_2
75,0.1795,0.1705
TEST 9 net 
75,0.7049,0.6885
>>>>5_2
TEST 5_2
128,0.3516,0.3305
TEST 9 net 
128,0.7049,0.6885
>>>>5_2
TEST 5_2
256,0.5797,0.5253
TEST 9 net 
256,0.7049,0.6885
>>>>5_2
TEST 9 net 
320,0.7049,0.6885
>>>>5_2
TEST 5_2
384,0.6505,0.5895
TEST 9 net 
384,0.7049,0.6885
>>>>5_3
TEST 5_3
75,0.1792,0.1716
TEST 9 net 
75,0.7049,0.6885
>>>>5_3
TEST 5_3
128,0.3318,0.3105
TEST 9 net 
128,0.7049,0.6885
>>>>5_3
TEST 5_3
256,0.5811,0.5032
TEST 9 net 
256,0.7049,0.6885
>>>>5_3
TEST 9 net 
320,0.7049,0.6885
>>>>5_3
TEST 9 net 
384,0.7049,0.6885
>>>>6_1
TEST 9 net 
75,0.7049,0.6885
>>>>6_1
TEST 9 net 
128,0.7049,0.6885
>>>>6_1
TEST 6_1
256,0.9992,0.8811
TEST 9 net 
256,0.7049,0.6885
